Descobrindo a senha com DEAP
============================



## Importações



In [1]:
import random
import numpy as np

from deap import base
from deap import creator
from deap import tools
from deap.algorithms import eaSimple

from funcoes import gene_letra

## Problema da senha usando `DEAP`



Primeiro definimos as constantes.



In [8]:
# relacionadas ao problema a ser resolvido
SENHA = "correcthorsebatterystaple"
LETRAS_POSSIVEIS = "abcdefghijklmnopqrstuvwxyz"
NUM_GENES = len(SENHA)

# relacionadas à busca
TAMANHO_POP = 50
NUM_GERACOES = 1000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.15
NUM_COMBATENTES_NO_TORNEIO = 3
TAMANHO_HALL_DA_FAMA = 1

Precisamos definir a função objetivo também.



In [9]:
def funcao_objetivo_senha(individuo, senha_verdadeira):
    """Computa a funcao objetivo de um individuo no problema da senha

    Args:
      individiuo: lista contendo as letras da senha
      senha_verdadeira: a senha que você está tentando descobrir

    Returns:
      Uma tupla com a  "distância" entre a senha proposta e a senha verdadeira.
      Essa distância é medida letra por letra. Quanto mais distante uma letra
      for da que deveria ser, maior é essa distância.
    """
    diferenca = 0

    for letra_candidato, letra_oficial in zip(individuo, senha_verdadeira):
        diferenca = diferenca + abs(ord(letra_candidato) - ord(letra_oficial))

    return (diferenca, )

O `DEAP` não tem uma função de mutação de letras, precisamos usar a que nós criamos na disciplina. Tem um pequeno porém, o `DEAP` espera que a função de mutação retorne um indivíduo dentro de uma tupla. Não sei dizer o motivo dessa escolha, temos que aceitar e seguir.



In [10]:
def mutacao_senha(individuo, letras):
    """Realiza a mutação de um gene no problema da senha.

    Args:
      individuo: uma lista representado um individuo no problema da senha
      letras: letras possíveis de serem sorteadas.

    Return:
      Uma tupla de um individuo (senha) com um gene mutado.
    """
    gene = random.randint(0, len(individuo) - 1)
    individuo[gene] = gene_letra(letras)
    return (individuo, )

O restante não tem segredo. Único detalhe é que agora o problema é de minimização, então precisamos alterar isso.



In [11]:
# Devemos alterar aqui por ser um problema de minimização
creator.create("Fitness_min", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.Fitness_min)

toolbox = base.Toolbox()

toolbox.register("cria_gene", gene_letra, LETRAS_POSSIVEIS)
toolbox.register(
    "individual",
    tools.initRepeat,
    creator.Individual,
    toolbox.cria_gene,
    NUM_GENES,
)
toolbox.register(
    "population", tools.initRepeat, list, toolbox.individual, TAMANHO_POP
)
toolbox.register("evaluate", funcao_objetivo_senha, SENHA)
toolbox.register(
    "select", tools.selTournament, tournsize=NUM_COMBATENTES_NO_TORNEIO
)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register(
    "mutate",
    mutacao_senha,
    letras=LETRAS_POSSIVEIS
)

hall_da_fama = tools.HallOfFame(TAMANHO_HALL_DA_FAMA)

estatisticas = tools.Statistics(lambda ind: ind.fitness.values)
estatisticas.register("avg", np.mean)
estatisticas.register("std", np.std)
estatisticas.register("min", np.min)
estatisticas.register("max", np.max)

populacao_inicial = toolbox.population()

populacao_final, log = eaSimple(
    populacao_inicial,
    toolbox,
    cxpb=CHANCE_CRUZAMENTO,
    mutpb=CHANCE_MUTACAO,
    ngen=NUM_GERACOES,
    stats=estatisticas,
    halloffame=hall_da_fama,
    verbose=True,
)

print()
print("Hall da fama:")
print("".join(hall_da_fama.items[0]))

gen	nevals	avg   	std    	min	max
0  	50    	220.66	26.8266	161	294
1  	31    	198.02	23.9186	151	265
2  	30    	178.38	18.2569	151	209
3  	31    	163.3 	10.1946	142	186
4  	25    	155.7 	7.9177 	141	173
5  	29    	149.08	7.02806	132	168
6  	31    	142.88	8.49856	122	169
7  	25    	135.64	7.9969 	120	158
8  	29    	130.14	10.1351	107	151
9  	34    	125.5 	8.89775	102	155
10 	38    	119.5 	6.38827	105	134
11 	31    	114.6 	5.77927	104	128
12 	25    	111.08	5.29845	99 	120
13 	31    	107.7 	4.08534	99 	116
14 	30    	105.12	5.1289 	94 	123
15 	26    	101.58	4.45012	92 	117
16 	38    	100.06	6.32269	92 	120
17 	37    	96.56 	5.55035	84 	113
18 	26    	94.48 	5.28485	84 	116
19 	37    	90.72 	3.48161	82 	98 
20 	25    	88.56 	5.94024	80 	109
21 	35    	84.88 	3.04394	76 	91 
22 	33    	82.66 	2.79005	78 	89 
23 	24    	81.54 	3.71597	78 	96 
24 	30    	81.22 	5.81477	76 	102
25 	31    	79.52 	4.29995	74 	101
26 	22    	77.54 	2.3256 	72 	84 
27 	29    	77.72 	6.42819	66 	98 
28 	24    	74.